In [1]:
from __future__ import print_function
import os, csv
from nltk.tag import StanfordPOSTagger
from nltk.parse.stanford import StanfordParser

In [2]:
st = StanfordPOSTagger('../Tools/stanford-postagger-full-2017-06-09/models/english-bidirectional-distsim.tagger','../Tools/stanford-postagger-full-2017-06-09/stanford-postagger-3.8.0.jar')
sp = StanfordParser(path_to_jar='../Tools/stanford-parser-full-2017-06-09/stanford-parser.jar',path_to_models_jar='../Tools/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar')

In [3]:
def extract_features(text):
    features = [{} for i in range(len(text.split()))]
    tags = st.tag(text.split())
    for i,tag in enumerate(tags):
        features[i][tag[1]] = 1
        
    return features

In [4]:
X = []
y = []
for root, dirs, files in os.walk('../Datasets/mpqa535/'):
    for f in files:
        with open(os.path.join(root,f)) as csvfile:
            reader = csv.reader(csvfile)
            reader.next()
            for row in reader:
                X.append(extract_features(row[0]))
                y.append(0 if row[1] == 'f' else 1)

In [18]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
X1 = [i for j in X for i in j]
vec.fit(X1)

# X1 = [np.array([vec.transform(j).toarray()[0] for j in i]) for i in X]
# X2 = np.asarray(X1)
X1 = np.array([np.array([np.where(vec.transform(j).toarray()[0] == 1)[0][0] + 1 for j in i]) for i in X])

In [20]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(X1)

Using TensorFlow backend.


In [24]:
labels = np.asarray(y)

In [27]:
VALIDATION_SPLIT = 0.2

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(data.max()+1, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15)

Epoch 1/15
8886/8886 [==============================] - 70s - loss: 0.6281 - acc: 0.6399    
Epoch 2/15
8886/8886 [==============================] - 55s - loss: 0.6038 - acc: 0.6763    
Epoch 3/15
8886/8886 [==============================] - 57s - loss: 0.6004 - acc: 0.6753    
Epoch 4/15
8886/8886 [==============================] - 61s - loss: 0.5982 - acc: 0.6753    
Epoch 5/15
8886/8886 [==============================] - 60s - loss: 0.5964 - acc: 0.6753    
Epoch 6/15
8886/8886 [==============================] - 54s - loss: 0.5951 - acc: 0.6817    
Epoch 7/15
8886/8886 [==============================] - 53s - loss: 0.5916 - acc: 0.6829    
Epoch 8/15
8886/8886 [==============================] - 52s - loss: 0.5894 - acc: 0.6829    
Epoch 9/15
8886/8886 [==============================] - 52s - loss: 0.5860 - acc: 0.6864    
Epoch 10/15
8886/8886 [==============================] - 52s - loss: 0.5847 - acc: 0.6880    
Epoch 11/15
8886/8886 [==============================] - 52s - loss: 

In [40]:
score, acc = model.evaluate(x_val, y_val,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

2208/2221 [============================>.] - ETA: 0sTest score: 0.622834729188
Test accuracy: 0.671769473344
